In [248]:
import pandas as pd
from scipy.signal import find_peaks
import plotly.graph_objects as go
import numpy as np
import itertools
# from peakdetect import peakdetect
import os
import matplotlib.pyplot as plt

In [249]:
# from google.colab import drive
# drive.mount('/content/drive')

In [250]:
# !ls "/content/drive/MyDrive/PHYS 243W/RLC Chaos Lab/data at 279.800kHz/*/*.05.csv"

In [251]:
storage_dict = {} # initalise dictonary to save keys = volts_strings and the corresponding_list

# Import data

In [252]:
folders = [0.24,   11.00 , 13.00 , 15.00  ,17.25 , 19.25 , 3.25	,5.50 , 7.75,
0.25 ,  11.25  ,13.25,  15.25,  1.75  , 19.50 , 3.50,	5.75 , 8.00,
0.50 ,  11.50 , 13.50 , 15.50  ,17.50  ,19.75  ,3.75	,6.00 ,8.25,
0.75  , 11.75  ,13.75  ,15.75 , 17.75,  19.99 , 4.00,	6.25,  8.50,
1.00  , 12.00,  14.00 , 16.00 , 18.00 , 2.00  , 4.25,	6.50 , 8.75,
10.00,  12.25 , 14.25,  16.25,  18.25,  2.25,   4.50,	6.75 , 9.00,
10.25,  1.25  , 14.50,  16.50 , 18.50 , 2.50,   4.75,	7.00 , 9.25,
10.50 , 12.50 , 14.75,  16.75,  18.75 , 2.75,   5.00,	7.25  ,9.50,
10.75 , 12.75 , 1.50  , 17.00  ,19.00 , 3.00,   5.25,	7.50,  9.75]

In [253]:
folders = np.sort(folders)
folders

array([ 0.24,  0.25,  0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ,
        2.25,  2.5 ,  2.75,  3.  ,  3.25,  3.5 ,  3.75,  4.  ,  4.25,
        4.5 ,  4.75,  5.  ,  5.25,  5.5 ,  5.75,  6.  ,  6.25,  6.5 ,
        6.75,  7.  ,  7.25,  7.5 ,  7.75,  8.  ,  8.25,  8.5 ,  8.75,
        9.  ,  9.25,  9.5 ,  9.75, 10.  , 10.25, 10.5 , 10.75, 11.  ,
       11.25, 11.5 , 11.75, 12.  , 12.25, 12.5 , 12.75, 13.  , 13.25,
       13.5 , 13.75, 14.  , 14.25, 14.5 , 14.75, 15.  , 15.25, 15.5 ,
       15.75, 16.  , 16.25, 16.5 , 16.75, 17.  , 17.25, 17.5 , 17.75,
       18.  , 18.25, 18.5 , 18.75, 19.  , 19.25, 19.5 , 19.75, 19.99])

In [254]:
path = "/Users/vashisth/Desktop/PHY_243W/data_at_279.800kHz/"
# file_path_ls = ['/content/drive/MyDrive/PHYS 243W/RLC Chaos Lab/data at 279.800kHz/{0:.2f}/{1:.2f}_05.csv'.format(x,x)  for x in folders]
file_path_ls = [path+'{0:.2f}/{1:.2f}_05.csv'.format(x,x)  for x in folders]
file_path_ls[28] = path+'7.00/7.00_005.csv'
file_path_ls[30] = path + '7.50/7.50_005.csv'
file_path_ls[0]

'/Users/vashisth/Desktop/PHY_243W/data_at_279.800kHz/0.24/0.24_05.csv'

In [255]:
def define_groups(l, threshold = 5):
    groups = {}
    its = 0
    for i in l:
        numbers_except = list(l)
        del numbers_except[(l.index(i))]
        for x in range(len(numbers_except)):
            if difference(i, numbers_except[x]) < threshold:
                try:
                    groups[its].append(numbers_except[x])
                except:
                    groups[its] = [numbers_except[x]]
                del l[l.index(numbers_except[x])]
        try:
            groups[its].append(i)
        except:
            groups[its] = [i]
        its += 1
    return(groups)

def difference(a, b):
    return abs(a-b)

In [256]:
def run_one(file_path, volt_value, threshold = 5):
  rlc_data = pd.read_csv(filepath_or_buffer =file_path) # first file

  if rlc_data.columns[0] =='Frequency': # had error from 16.50 onwards
    rlc_data = rlc_data.rename(columns = {'Frequency': 'Time'})

  time_values = rlc_data ['Time'].to_numpy()  [1:].astype(float)
  time_series_A = rlc_data['Channel A'].to_numpy() [1:].astype(float)
  time_series_B = rlc_data['Channel B'].to_numpy() [1:].astype(float)

  #indices = find_peaks(time_series_B,  prominence=1 )[0]
  # indices = find_peaks(time_series_B,  prominence=0, distance=110)[0]
  indices = find_peaks(time_series_B,  prominence=0, distance=2.5)[0]


  peaks_channel_A =  [time_series_A[j] for j in indices]
  peaks_channel_B =  [time_series_B[j] for j in indices]

  tuples_A_B = list( zip(peaks_channel_A, peaks_channel_B) )
  tuples_A_B = sorted(tuples_A_B, key=lambda x: x[0])

  dict_groups = define_groups(sorted( list( peaks_channel_A) ), threshold=threshold )
  keys = list ( dict_groups.keys() )

  peaks_channel_A_unique = [ i for i in peaks_channel_A ]
  for i in range( len(peaks_channel_A) ) :
      for j in keys:
          if peaks_channel_A_unique[i] in dict_groups[j]:
              peaks_channel_A_unique[i] = np.mean(dict_groups[j])
          else:
              continue

  tuples_A_B_rounded = list( zip(peaks_channel_A_unique, peaks_channel_B) )
  tuples_A_B_rounded = sorted(tuples_A_B_rounded, key=lambda x: x[0])
  
  storage_dict = {}
  volts_string = str(volt_value)
  corresponding_list = [next(g) for _, g in itertools.groupby(sorted(tuples_A_B_rounded))]
  
  storage_dict[volts_string] = corresponding_list
  return storage_dict

In [257]:
dict = run_one(file_path_ls[0],folders[0])
dict

{'0.24': [(0.012065332758620687, -120.2063),
  (0.012065332758620687, -120.0171),
  (0.012065332758620687, -119.9683),
  (0.012065332758620687, -119.8889),
  (0.012065332758620687, -119.8401),
  (0.012065332758620687, -119.8279),
  (0.012065332758620687, -119.7485),
  (0.012065332758620687, -119.7119),
  (0.012065332758620687, -119.541),
  (0.012065332758620687, -119.5227),
  (0.012065332758620687, -119.3518),
  (0.012065332758620687, -119.3152),
  (0.012065332758620687, -119.2053),
  (0.012065332758620687, -119.1748),
  (0.012065332758620687, -119.1626),
  (0.012065332758620687, -119.0771),
  (0.012065332758620687, -118.7536),
  (0.012065332758620687, -118.6621),
  (0.012065332758620687, -118.54),
  (0.012065332758620687, -118.4118),
  (0.012065332758620687, -117.9357),
  (0.012065332758620687, 114.1453),
  (0.012065332758620687, 115.1097),
  (0.012065332758620687, 115.5064),
  (0.012065332758620687, 115.5309),
  (0.012065332758620687, 115.7323),
  (0.012065332758620687, 115.7567),
  

In [258]:
# def run_one(file_path, volt_value, threshold = 5):
#   rlc_data = pd.read_csv(filepath_or_buffer =file_path) # first file

#   if rlc_data.columns[0] =='Frequency': # had error from 16.50 onwards
#     rlc_data = rlc_data.rename(columns = {'Frequency': 'Time'})

#   time_values = rlc_data ['Time'].to_numpy()  [1:].astype(float)
#   time_series_A = rlc_data['Channel A'].to_numpy() [1:].astype(float)
#   time_series_B = rlc_data['Channel B'].to_numpy() [1:].astype(float)

#   #indices = find_peaks(time_series_B,  prominence=1 )[0]
#   # indices = find_peaks(time_series_B,  prominence=0, distance=110)[0]
#   indices = find_peaks(time_series_B,  prominence=0, distance=2.6, threshold=threshold)[0]


#   peaks_channel_A =  [time_series_A[j] for j in indices]
#   peaks_channel_B =  [time_series_B[j] for j in indices]

#   tuples_A_B = list( zip(peaks_channel_A, peaks_channel_B) )
#   tuples_A_B = sorted(tuples_A_B, key=lambda x: x[0])

#   # dict_groups = define_groups(sorted( list( peaks_channel_A) ), threshold=1)
#   # keys = list ( dict_groups.keys() )

#   # peaks_channel_A_unique = [ i for i in peaks_channel_A ]
#   # for i in range( len(peaks_channel_A) ) :
#   #     for j in keys:
#   #         if peaks_channel_A_unique[i] in dict_groups[j]:
#   #             peaks_channel_A_unique[i] = np.mean(dict_groups[j])
#   #         else:
#   #             continue

#   # tuples_A_B_rounded = list( zip(peaks_channel_A_unique, peaks_channel_B) )
#   tuples_A_B_rounded = sorted(tuples_A_B, key=lambda x: x[0])
  
#   storage_dict = {}
#   volts_string = str(volt_value)
#   corresponding_list = [next(g) for _, g in itertools.groupby(sorted(tuples_A_B_rounded))]
  
#   storage_dict[volts_string] = corresponding_list
#   return storage_dict

In [259]:
def run_all(ls_path, ls_volts, threshold):
  store_all = {}
  for i in range(len(ls_path)):
    path = ls_path[i]
    volt_value = ls_volts[i]
    store_one = run_one(file_path = path, volt_value = volt_value, threshold = threshold)
    store_all.update(store_one)
  
  return store_all

In [260]:
file_path_ls[28]

'/Users/vashisth/Desktop/PHY_243W/data_at_279.800kHz/7.00/7.00_005.csv'

In [261]:
run_one(file_path_ls[66],folders[66])

{'16.5': [(-40.4217625, -7.853633),
  (-40.4217625, -7.834101),
  (-40.4217625, -7.830134),
  (-40.4217625, -7.772149),
  (-34.06425, -7.850886),
  (-34.06425, -7.837459),
  (-34.06425, -7.812433),
  (-34.06425, -7.800836),
  (-34.06425, -7.766961),
  (-34.06425, -7.758721),
  (-20.153406666666665, -7.850581),
  (-20.153406666666665, -7.809992),
  (-20.153406666666665, -7.805719),
  (-15.059356666666666, -7.85699),
  (-15.059356666666666, -7.847529),
  (-15.059356666666666, -7.82403),
  (-15.059356666666666, -7.812433),
  (-15.059356666666666, -7.769707),
  (-15.059356666666666, -7.754753),
  (-5.575731333333334, -7.866451),
  (-5.575731333333334, -7.83227),
  (-5.575731333333334, -7.78161),
  (-0.55238525, -7.847529),
  (-0.55238525, -7.804804),
  (6.839198, 7.874386),
  (14.601071666666664, 7.850276),
  (14.601071666666664, 7.919248),
  (14.601071666666664, 7.944883),
  (14.601071666666664, 7.97235),
  (14.601071666666664, 7.989135),
  (14.601071666666664, 7.98944),
  (18.894616, 7.8

In [262]:
dic = run_all(file_path_ls,folders, threshold=5)

In [268]:
keys, vals , vals_len= [], [], []
for k in dic.keys():
  keys.append(k)
  vals.append(dic.get(k))
  vals_len.append(len(dic.get(k)))

In [269]:
i = 0
vals[i][0] 

plt_lsx = []

for j in range(len(vals)):
  ls = [vals[j][i][0] for i in range(len(vals[j])) ]
  for k in range(len(ls)):
    plt_lsx.append([float(keys[j]), ls[k]])


x_axis = [i[0] for i in plt_lsx]  
y_axis = np.asarray([i[1] for i in plt_lsx] ) 

In [271]:
# this is when you have MV
y_axis [0:4] = y_axis [0:4] *( 1/1000)
y_axis

array([1.20653328e-05, 1.20653328e-05, 1.20653328e-05, ...,
       7.60175367e+01, 7.60175367e+01, 7.60175367e+01])

In [272]:
trace1 = go.Scatter(
    x = x_axis[5:],
    y = y_axis[5:],
    mode='markers ',   # scatter mode (more in UG section 1)
    # text = label3,
    opacity = 1,
    # textposition='top center',
    # marker = dict(size = 12, color = 'black', line=dict(width=0.5))
    )
data = [trace1]

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,

    xaxis= go.layout.XAxis(linecolor = 'black',
                          linewidth = 1,
                          mirror = True),

    yaxis= go.layout.YAxis(linecolor = 'black',
                          linewidth = 1,
                          mirror = True),

    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad = 4
    )
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(title= str(50) + 'threshold' )
fig.update_layout(xaxis_title="Output Voltage (mV)", yaxis_title="Driving Voltage (V)")

fig.show()

In [273]:
threshold = [0+ 1* i for i in range(0,15)]
for distance in threshold: 
    dic = run_all(file_path_ls,folders, threshold= distance)
    keys, vals , vals_len= [], [], []
    for k in dic.keys():
        keys.append(k)
        vals.append(dic.get(k))
        vals_len.append(len(dic.get(k)))
        # print(k + "  " + str(len(dic.get(k))) )
        
    i = 0
    vals[i][0] 
    plt_lsx = []

    for j in range(len(vals)):
        ls = [vals[j][i][0] for i in range(len(vals[j])) ]
        for k in range(len(ls)):
            plt_lsx.append([float(keys[j]), ls[k]])

    x_axis = [i[0] for i in plt_lsx]  
    y_axis = np.asarray([i[1] for i in plt_lsx] ) 
    
    # this is when you have MV
    y_axis [0:4] = y_axis [0:4] *( 1/1000)
    y_axis

    trace1 = go.Scatter(
        x = x_axis[5:],
        y = y_axis[5:],
        mode='markers ',   # scatter mode (more in UG section 1)
        opacity = .6,
        # marker = dict(size = 8, color = 'blue', line=dict(width=0.5))
        )
    data = [trace1]

    layout = go.Layout(
        autosize=False,
        width=750,
        height=500,

        xaxis= go.layout.XAxis(linecolor = 'black',
                            linewidth = 1,
                            mirror = True),

        yaxis= go.layout.YAxis(linecolor = 'black',
                            linewidth = 1,
                            mirror = True),

        margin=go.layout.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad = 4
        )
    )

    fig = go.Figure(data=data, layout=layout)
    fig.update_layout(title= 'threshold =' + str(distance) )
    fig.update_layout(yaxis_title="Output Voltage (mV)", xaxis_title="Driving Voltage (V)")
    fig.show()

In [73]:
 # Check
vals[60]

[(17.14428, 7.256386),
 (17.14428, 7.258828),
 (17.14428, 7.260964),
 (22.980436666666666, 7.248756),
 (22.980436666666666, 7.260659),
 (32.64666666666667, 7.250893),
 (32.64666666666667, 7.253029),
 (32.64666666666667, 7.25486),
 (35.92028, 7.242042),
 (48.97711999999999, 7.237159),
 (48.97711999999999, 7.248756),
 (48.97711999999999, 7.249367),
 (57.4694, 7.238685)]

In [74]:
rlc_data_1 = pd.read_csv(filepath_or_buffer = path+'16.50/16.50_05.csv') # first file

#rlc_data_2 = pd.read_csv(filepath_or_buffer ='/content/drive/MyDrive/PHYS 243W/RLC Chaos Lab/data at 279.800kHz/0.24/0.24_06.csv') # second file
#rlc_data_2 = rlc_data_2.drop(axis=0, index=0)

In [75]:

#rlc_data = rlc_data_1.append(rlc_data_2, ignore_index=True)
rlc_data = rlc_data_1

In [48]:
if rlc_data.columns[0] =='Frequency':
  rlc_data = rlc_data.rename(columns = {'Frequency': 'Time'})

In [49]:
rlc_data

,Time,Channel A,Channel B
0,(us),(mV),(V)
1,-25.09928677,-14.60616000,1.54240500
2,-25.07428677,-13.69060000,1.89794600
3,-25.04928677,-12.78725000,2.19885800
4,-25.02428677,-11.75268000,2.51289400
...,...,...,...
2012,25.17571025,2.45368800,4.54725800
2013,25.20071025,3.07626600,4.85702100
2014,25.22571025,3.69579100,5.17014100
2015,25.25071024,4.23902100,5.40360700


In [50]:
time_values = rlc_data ['Time'].to_numpy()  [1:].astype(float)
time_series_A = rlc_data['Channel A'].to_numpy() [1:].astype(float)
time_series_B = rlc_data['Channel B'].to_numpy() [1:].astype(float)

## Draw from sanity check 

In [51]:
fig = go.Figure(data=go.Scatter(
    x = time_values,
    y = time_series_B,
    mode = 'markers'
))

fig.show()

# Find and draw peaks

In [52]:
indices = find_peaks(time_series_B,  prominence=0, distance=110)[0]
indices

array([  30,  174,  317,  461,  602,  746,  889, 1033, 1174, 1318, 1461,
       1605, 1746, 1890])

In [53]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=time_series_B,
    mode='lines+markers',
    name='Channel A'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_B[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks (A)'
))

fig.show()

## Drawing peaks of A and corresponding values of channel B

In [54]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=time_series_A,
    mode='lines+markers',
    name='Channel A'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_A[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks (A)'
))

fig.add_trace(go.Scatter(
    y=time_series_B,
    mode='lines+markers',
    name='Channel B'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_B[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='circle'
    ),
    name='B values corr to Peak A'
))

fig.show()

# Getting the tuple of the values of peaks

In [55]:
peaks_channel_A =  [time_series_A[j] for j in indices]
peaks_channel_B =  [time_series_B[j] for j in indices]

In [56]:
tuples_A_B = list( zip(peaks_channel_A, peaks_channel_B) )
tuples_A_B = sorted(tuples_A_B, key=lambda x: x[0])
tuples_A_B

[(15.40574, 7.97235),
 (16.27247, 7.98944),
 (16.35487, 7.989135),
 (17.76177, 7.977539),
 (21.98858, 7.980896),
 (23.1727, 7.978454),
 (27.94275, 7.973266),
 (28.44935, 7.982421),
 (29.15128, 7.975707),
 (46.04633, 7.96289),
 (46.87948, 7.975707),
 (64.391, 7.965026),
 (66.00848, 7.966552),
 (66.93014, 7.957396)]

# Get unique ones 
Notice that 62.5 and 63 are basically the same peak but we cannot just take the unique set because these two would come as different values. So we first write a function where the values are clubbed together if they are within a certain threshold

In [57]:
def define_groups(l, threshold = 5):
    groups = {}
    its = 0
    for i in l:
        numbers_except = list(l)
        del numbers_except[(l.index(i))]
        for x in range(len(numbers_except)):
            if difference(i, numbers_except[x]) < threshold:
                try:
                    groups[its].append(numbers_except[x])
                except:
                    groups[its] = [numbers_except[x]]
                del l[l.index(numbers_except[x])]
        try:
            groups[its].append(i)
        except:
            groups[its] = [i]
        its += 1
    return(groups)

def difference(a, b):
    return abs(a-b)

In [58]:
dict_groups = define_groups(sorted( list( peaks_channel_A) ) )
dict_groups

{0: [16.27247, 16.35487, 17.76177, 15.40574],
 1: [23.1727, 21.98858],
 2: [28.44935, 29.15128, 27.94275],
 3: [46.87948, 46.04633],
 4: [66.00848, 66.93014, 64.391]}

### if they are in the same group replace by the mean

In [59]:
peaks_channel_A[1]

64.391

In [60]:
keys = list ( dict_groups.keys() )

In [61]:
peaks_channel_A_unique = [ i for i in peaks_channel_A ]
for i in range( len(peaks_channel_A) ) :
    for j in keys:
        # print(j)
        # print( peaks_channel_A[i] )
        # print(j)
        if peaks_channel_A_unique[i] in dict_groups[j]:
            peaks_channel_A_unique[i] = np.mean(dict_groups[j])
        else:
            continue

In [62]:
print( "original: ", peaks_channel_A)
print( "unique: ", peaks_channel_A_unique)

original:  [15.40574, 64.391, 28.44935, 23.1727, 46.04633, 16.27247, 66.00848, 29.15128, 17.76177, 46.87948, 16.35487, 66.93014, 27.94275, 21.98858]
unique:  [16.4487125, 65.77654000000001, 28.51446, 22.58064, 46.462905, 16.4487125, 65.77654000000001, 28.51446, 16.4487125, 46.462905, 16.4487125, 65.77654000000001, 28.51446, 22.58064]


In [63]:
tuples_A_B_rounded = list( zip(peaks_channel_A_unique, peaks_channel_B) )
tuples_A_B_rounded = sorted(tuples_A_B_rounded, key=lambda x: x[0])
tuples_A_B_rounded

[(16.4487125, 7.97235),
 (16.4487125, 7.98944),
 (16.4487125, 7.977539),
 (16.4487125, 7.989135),
 (22.58064, 7.978454),
 (22.58064, 7.980896),
 (28.51446, 7.982421),
 (28.51446, 7.975707),
 (28.51446, 7.973266),
 (46.462905, 7.96289),
 (46.462905, 7.975707),
 (65.77654000000001, 7.965026),
 (65.77654000000001, 7.966552),
 (65.77654000000001, 7.957396)]

### finally group the tuples with the same first coordinate together

In [64]:
f = lambda x:x[0]
x = [next(g) for _, g in itertools.groupby(sorted(tuples_A_B_rounded, key=f), key=f)]
x

[(16.4487125, 7.97235),
 (22.58064, 7.978454),
 (28.51446, 7.982421),
 (46.462905, 7.96289),
 (65.77654000000001, 7.965026)]

In [65]:
volts_string = '7 Volts'
corresponding_list = [next(g) for _, g in itertools.groupby(sorted(tuples_A_B_rounded, key=f), key=f)]

In [66]:
print("Number of points for the bifurication diagram are : {}".format(len(x)))

Number of points for the bifurication diagram are : 5


## Getting dictionary

In [67]:
storage_dict[volts_string] = corresponding_list

In [68]:
storage_dict

{'7 Volts': [(16.4487125, 7.97235),
  (22.58064, 7.978454),
  (28.51446, 7.982421),
  (46.462905, 7.96289),
  (65.77654000000001, 7.965026)]}